In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import sklearn
import os
import time
import warnings

warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd
from sklearn.decomposition import PCA
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split

import xgboost
import lightgbm

#模型集成
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier

#模型调节
from sklearn.model_selection import GridSearchCV #参数搜索
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']


# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 2
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



In [4]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

def load_data(filename):    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    #截掉前后数据
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate]
    
    #滤波
    #return butter_worth(data , 0.5 , 30)

    return data

In [5]:
def separate(data , label , overlap_length = 128):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [6]:
def shuffle_t_v(filenames):
    np.random.shuffle(filenames)
    
    return filenames

def combine(freq):
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('data/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('data/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('data/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('data/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('data/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('data/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('data/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('data/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('data/8/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('data/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('data/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('data/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('data/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('data/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('data/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('data/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('data/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('data/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
        
    #============
    #训练数据分段
    person_data_0 , person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    person_data_1 , person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    person_data_2 , person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    person_data_3 , person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    person_data_4 , person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    person_data_5 , person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    person_data_6 , person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    person_data_7 , person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    person_data_8 , person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    data = np.concatenate((person_data_0 , person_data_1 , person_data_2 ,
                           person_data_3 , person_data_4 , person_data_5 ,
                           person_data_6 , person_data_7 , person_data_8 
                        ))
    
    labels = np.concatenate((person_labels_0 , person_labels_1 , person_labels_2 ,
                             person_labels_3 , person_labels_4 , person_labels_5 ,
                             person_labels_6 , person_labels_7 , person_labels_8 
                        ))
    
    '''
    data维度 样本数量 , 256*3 , 16通道
    '''
    return data , labels

# FC-AE

In [10]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input

In [11]:
encoder_input = Input(shape = (256*3*16 , ))

#encoder
encoder = Dense(units=512 , activation='relu')(encoder_input)
encoder = Dense(units=256 , activation='relu')(encoder)
encoder = Dense(units=128 , activation='relu')(encoder)
encoder_output = Dense(units=64 , activation='relu')(encoder) #聚类需要使用的2维特征

#decoder
decoder = Dense(units=128 , activation='relu')(encoder_output)
decoder = Dense(units=256 , activation='relu')(decoder)
decoder = Dense(units=512 , activation='relu')(decoder)
decoder_output = Dense(units=256*3*16 , activation='relu')(decoder)


In [12]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# stacking  
# svm knn 'naive bayes' 'decision tree'  

In [13]:
acc_s = []

estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]

for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.reshape(data , newshape=(-1 , 768*16))
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.9403183023872679
0.9482758620689655
0.9323607427055703
0.9350132625994695
0.9416445623342176
0.9442970822281167
0.9403183023872679
0.946949602122016
0.9429708222811671
0.9522546419098143
0.9442970822281167
0.946949602122016
0.9535809018567639
0.9297082228116711
0.9708222811671088
0.9429708222811671
0.9615384615384616
0.9496021220159151
0.9482758620689655
0.9442970822281167
freq =  6 average_acc =  0.9458222811671086 var_acc =  8.307154064265566e-05
freq =  7.5
0.9694960212201591
0.9482758620689655
0.9681697612732095
0.9522546419098143
0.9562334217506632
0.9694960212201591
0.9496021220159151
0.9655172413793104
0.9748010610079576
0.9535809018567639
0.9694960212201591
0.9509283819628647
0.9641909814323607
0.9562334217506632
0.9681697612732095
0.9641909814323607
0.9681697612732095
0.9575596816976127
0.9615384615384616
0.9655172413793104
freq =  7.5 average_acc =  0.9616710875331564 var_acc =  5.978723553954499e-05
freq =  8.5
0.9734748010610079
0.9615384615384616
0.980106100795

# CNN-AE

In [7]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input , Deconv2D , UpSampling2D

Using TensorFlow backend.


In [8]:
encoder_input = Input(shape = (256*3 , 16 , 1))
#encoder
#VGG16结构
encoder = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder_input)
encoder = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
encoder = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder = MaxPooling2D(pool_size=(2, 2))(encoder)
#new_add
encoder_output = Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(encoder)
encoder_output = MaxPooling2D(pool_size=(2, 2))(encoder_output)
#shape = encoder_output.get_shape().as_list()

encoder_output = Flatten()(encoder_output)
encoder_output = Dense(128 , activation='relu')(encoder_output)
encoder_output = Dense(64 , activation='relu')(encoder_output)

#decoder
decoder = Dense(128 , activation='relu')(encoder_output)
decoder = Dense(48*1*32 , activation='relu')(decoder)
decoder = Reshape(target_shape=(48,1,32))(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = Deconv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder = UpSampling2D(size=(2,2))(decoder)
decoder = Deconv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)
decoder_output = Deconv2D(1, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform')(decoder)


In [9]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# stacking  
# svm knn 'naive bayes' 'decision tree'  
# 4种频率 显存太小

In [10]:
acc_s = []
estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]
for freq in [6]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.916445623342175
0.9045092838196287
0.916445623342175
0.9005305039787799
0.9111405835543767
0.8912466843501327
0.9204244031830239
0.916445623342175
0.8885941644562334
0.9084880636604774
0.8859416445623343
0.9137931034482759
0.9018567639257294
0.9310344827586207
0.9151193633952255
0.9111405835543767
0.916445623342175
0.9018567639257294
0.9071618037135278
0.8938992042440318
freq =  6 average_acc =  0.9076259946949603 var_acc =  0.00012792515953816586


In [10]:
acc_s = []
estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]
for freq in [7.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  7.5
0.9920424403183024
0.9854111405835544
0.9827586206896551
0.9907161803713528
0.9880636604774535
0.9854111405835544
0.9854111405835544
0.9827586206896551
0.9854111405835544
0.9840848806366048
0.9827586206896551
0.9814323607427056
0.9880636604774535
0.9854111405835544
0.9907161803713528
0.9920424403183024
0.9907161803713528
0.986737400530504
0.9840848806366048
0.9854111405835544
freq =  7.5 average_acc =  0.9864721485411142 var_acc =  1.0307537518732935e-05


In [10]:
acc_s = []
estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]
for freq in [8.5]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  8.5
0.9655172413793104
0.9708222811671088
0.9774535809018567
0.9814323607427056
0.9774535809018567
0.9854111405835544
0.9721485411140584
0.9761273209549072
0.9761273209549072
0.980106100795756
0.9734748010610079
0.9840848806366048
0.9827586206896551
0.9694960212201591
0.9840848806366048
0.9827586206896551
0.9694960212201591
0.9840848806366048
0.9761273209549072
0.9814323607427056
freq =  8.5 average_acc =  0.9775198938992042 var_acc =  3.2624411626058113e-05


In [10]:
acc_s = []
estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]
for freq in [10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = np.expand_dims(data , axis=-1)
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  10
0.9854111405835544
0.9840848806366048
0.986737400530504
0.9814323607427056
0.9814323607427056
0.9774535809018567
0.9840848806366048
0.9814323607427056
0.9827586206896551
0.9907161803713528
0.980106100795756
0.9787798408488063
0.9840848806366048
0.980106100795756
0.9880636604774535
0.9827586206896551
0.9814323607427056
0.980106100795756
0.9880636604774535
0.9814323607427056
freq =  10 average_acc =  0.983023872679045 var_acc =  1.1011123697486057e-05


# 结束
# ==

# RNN-AE

In [7]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input , LSTM

Using TensorFlow backend.


In [8]:
encoder_input = Input(shape = (256*3 , 16))

#encoder
encoder = LSTM(32 , return_sequences=True)(encoder_input)
encoder_output = Dense(64 , activation='relu')(encoder)
encoder_output = Flatten()(encoder_output)
encoder_output = Dense(64 , activation='relu')(encoder_output)

#decoder
decoder = Dense(768*64 , activation='relu')(encoder_output)
decoder = Reshape(target_shape=(768 , 64))(decoder)
decoder = LSTM(32)(decoder)
decoder_output = Dense(256*3*16 , activation='relu')(decoder)
decoder_output = Reshape(target_shape=(256*3 , 16))(decoder_output)

In [9]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

# stacking  
# svm knn 'naive bayes' 'decision tree'  

In [10]:
acc_s = []
estimators = [
    ('svm', SVC()),
    ('knn', KNeighborsClassifier()),
    ('bayes' , GaussianNB()),
    ('decision' , DecisionTreeClassifier()),
            ]
for freq in [6 , 7.5 , 8.5 , 10]:
    print('freq = ' , freq)
    acc_s = []
    
    data , labels = combine(freq) #读取数据
    data = (data-np.mean(data)) / np.std(data)
    
    autoencoder.compile(optimizer='adam' , loss = 'mse')
    autoencoder.fit(data , data , batch_size=32 , epochs=5 , verbose=0 , validation_split=0.1)
    
    data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
    for t in range(20): #循环20次 取平均值 作为最终结果
        classifier = StackingClassifier(estimators=estimators , final_estimator = RandomForestClassifier(n_estimators=20))

        train_X , val_X , train_y , val_y = train_test_split(data_feature , labels , test_size = 0.1)

        classifier.fit(train_X , train_y)

        acc = classifier.score(val_X , val_y)
        acc_s.append(acc)
        print(acc)
        
    print('freq = ' , freq , 'average_acc = ' , np.average(acc_s) , 'var_acc = ' , np.var(acc_s))

freq =  6
0.9734748010610079
0.9748010610079576
0.9761273209549072
0.9509283819628647
0.9814323607427056
0.9761273209549072
0.9588859416445623
0.980106100795756
0.980106100795756
0.9748010610079576
0.9681697612732095
0.9787798408488063
0.9708222811671088
0.9641909814323607
0.9721485411140584
0.9734748010610079
0.9681697612732095
0.9708222811671088
0.9721485411140584
0.9721485411140584
freq =  6 average_acc =  0.9718832891246686 var_acc =  5.146732897578981e-05
freq =  7.5
0.9827586206896551
0.9774535809018567
0.9827586206896551
0.9774535809018567
0.986737400530504
0.9827586206896551
0.9827586206896551
0.9907161803713528
0.980106100795756
0.9854111405835544
0.9840848806366048
0.9907161803713528
0.9748010610079576
0.986737400530504
0.9814323607427056
0.9814323607427056
0.9880636604774535
0.9840848806366048
0.9893899204244032
0.9880636604774535
freq =  7.5 average_acc =  0.9838859416445624 var_acc =  1.8693405286746504e-05
freq =  8.5
0.9907161803713528
0.9840848806366048
0.98541114058355